In [ ]:
!pip install --upgrade pip
!pip install imblearn

In [ ]:
!pip install Scikit-learn

In [ ]:
!pip install Matplotlib

In [ ]:
!pip install scikit-plot

In [ ]:
import pandas as pd
import numpy
import seaborn
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression  #l逻辑回归
from sklearn.ensemble import RandomForestClassifier   #随机森林
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn import metrics 
import scikitplot 
#Hide all the warnings in jupyter notebook
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
# 解压数据
!unzip /home/aistudio/data/data190996/MachineLearningCSV.zip -d ./data58344/
!unzip /home/aistudio/data/data190996/GeneratedLabelledFlows.zip -d ./data/

In [ ]:
# 合并数据集
'''
data0 = pd.read_csv('data58344/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',  low_memory = False)
data1 = pd.read_csv('data58344/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', low_memory = False)
data2 = pd.read_csv('data58344/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv',  low_memory = False)
data3 = pd.read_csv('data58344/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv',  low_memory = False)
data4 = pd.read_csv('data58344/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', low_memory = False)
data5 = pd.read_csv('data58344/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',  low_memory = False)
data6 = pd.read_csv('data58344/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv', low_memory = False)
data7 = pd.read_csv('data58344/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv', low_memory = False)
data_total = pd.concat([data0,data1,data2,data3,data4,data5,data6,data7])
data_total.to_csv("X_combine.csv",index=False)
'''

In [ ]:
import pandas as pd
#n = 100000  # 随机选择10行
filename = "X_combine.csv"  # 文件名

data_total = pd.read_csv(filename)
#data_total = data_total.sample(n=n)
print(data_total)

In [ ]:
data_total.info()

In [ ]:
data_total.shape

In [ ]:
#统计缺失值数据

data_total.isnull().sum()

data_total = data_total.dropna() 

In [ ]:
data_total.shape

In [ ]:
# 无穷值或者空值处理  空值替换INF 最后删除
data_total = data_total.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [ ]:
data_total.shape

In [ ]:
data_total.columns

In [ ]:
data_total.to_csv("Merge_2017_clean.csv",index=False)

In [ ]:
# 检查每列的值是否相同
to_drop = []
for column in data_total.columns:
    if data_total[column].nunique() == 1:
        to_drop.append(column)

# 删除需要删除的列
data_total.drop(columns=to_drop, inplace=True)


**划分数据**

In [ ]:
# 查看数据结构，划分数据
print(data_total.shape)
df=data_total
X = df.loc[:,df.columns != " Label"]
Y = df.loc[:,df.columns == " Label"]
print(X.shape)
print(Y.shape)

In [ ]:
import seaborn as sns

In [ ]:
# 查看数据分布
def ClassDistribution(y):
    ax = sns.countplot(y[' Label'],label="Count")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    print(y.groupby(' Label').size())

ClassDistribution(Y)


In [ ]:
# 无穷值或者空值处理
import numpy as np
X.fillna(0, inplace=True)
train_X = np.isinf(X)
X[train_X]=0

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, train_size = 0.8, test_size = 0.2, random_state = 42) #shuffle=False

In [ ]:
# 调用模型进行训练，换其他模型改这里就行
xg = xgb.XGBClassifier(n_estimators = 10)
xg.fit(X_train,y_train)
xg_score=xg.score(X_test,y_test)
y_predict=xg.predict(X_test)
y_true=y_test
print('Accuracy of XGBoost: '+ str(xg_score))
precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted') 
print('Precision of XGBoost: '+(str(precision)))
print('Recall of XGBoost: '+(str(recall)))
print('F1-score of XGBoost: '+(str(fscore)))
print(classification_report(y_true,y_predict))
cm=confusion_matrix(y_true,y_predict)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

**以下是降维方法**

**1.0 SelectKBest**

In [ ]:
#选择出前k个与标签最相关的特征。
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=30)
selector.fit(X, Y)

selected_features = X.columns[selector.get_support()]
selected_features

**2.0  互信息分类**

In [ ]:
import pandas as pd
n = 5000  # 随机选择10行
filename = "X_combine.csv"  # 文件名

data_sample = pd.read_csv(filename)
data_sample = data_sample.sample(n=n)
print(data_sample)

In [ ]:
data_sample.shape

In [ ]:
#统计缺失值数据
data_sample.isnull().sum()
data_sample = data_sample.dropna() 

In [ ]:
# 检查每列的值是否相同
to_drop_two= []
for column in data_sample.columns:
    if data_sample[column].nunique() == 1:
        to_drop_two.append(column)

# 删除需要删除的列
data_sample.drop(columns=to_drop_two, inplace=True)


In [ ]:
#更新后的数据维度
data_sample.shape

In [ ]:
data_sample[' Label'].value_counts()

In [ ]:
# 查看数据结构，划分数据
df_two=data_sample
X_two = df_two.loc[:,df_two.columns != " Label"]
Y_two = df_two.loc[:,df_two.columns == " Label"]
print(X_two.shape)
print(Y_two.shape)
# 无穷值或者空值处理
import numpy as np
X_two.fillna(0, inplace=True)
train_X_two = np.isinf(X_two)
X_two[train_X_two]=0
X_two.shape
Y_two.shape



In [ ]:
df_two.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_two[' Label'] = le.fit_transform(df_two[' Label']).astype(np.int64)

In [ ]:
#统计
df_two[' Label'].value_counts()


In [ ]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X_two, Y_two)#互信息分类

In [ ]:

# Min-max normalization
numeric_features = df_two.dtypes[df_two.dtypes != 'object'].index
df_two[numeric_features] = df_two[numeric_features].apply(
    lambda x: (x - x.min()) / (x.max()-x.min()))
# Fill empty values by 0
df_two = df_two.fillna(0)

'''
# Z-score normalization
numeric_features = df_two.dtypes[df_two.dtypes != 'object'].index
df_two[numeric_features] = df_two[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# Fill empty values by 0
df_two = df_two.fillna(0)
'''


In [ ]:
# calculate the sum of importance scores
#  计算重要性分数的总和
f_list = sorted(zip(map(lambda x: round(x, 4), importances), numeric_features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])
    
# select the important features from top to bottom until the accumulated importance reaches 90%
# 从上到下选择重要特征，直到累积重要性达到90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), numeric_features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.90:
        break


In [ ]:
X_fs = df_two[fs]
X_fs.columns

In [ ]:
len(X_fs.columns)

In [ ]:
#X_three = df_three.drop([' Label'],axis=1)
#Y_three = df_three[' Label']
#X_train_three, X_test_three, Y_train_three, Y_test_three = train_test_split(X_three,Y_three, train_size = 0.8, test_size = 0.2, random_state = 42) #shuffle=False
X_two = df_two.loc[:,df_two.columns != " Label"]
Y_two = df_two.loc[:,df_two.columns == " Label"]
X_train_two, X_test_two, Y_train_two, Y_test_two = train_test_split(X_two,Y_two, train_size = 0.8, test_size = 0.2, random_state = 42) #shuffle=False

In [ ]:
Y_train_two.value_counts()

In [ ]:
#调用随机森林模型训练
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# 定义随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=123)
# 在训练集上拟合模型
train_model_rf = rf_model.fit(X_two, Y_two)
# 计算准确率
print("Score:", rf_model.score(X_two,Y_two))

3.0

In [ ]:
import copy
df_three=copy.deepcopy(df_two)

In [ ]:
X_three = df_three.drop([' Label'],axis=1)
Y_three = df_three[' Label']
X_train_three, X_test_three, Y_train_three, Y_test_three = train_test_split(X_three,Y_three, train_size = 0.8, test_size = 0.2, random_state = 42) #shuffle=False

In [ ]:
#object编码  train['labels']
'''
from sklearn.preprocessing import LabelEncoder
label_encoding = LabelEncoder()
train['labels'] = label_encoding.fit_transform()
train['protocol_type'] = label_encoding.fit_transform(train['protocol_type'])
train['service'] = label_encoding.fit_transform(train['service'])
train['flag'] = label_encoding.fit_transform(train['flag'])
'''
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_three[' Label'] = le.fit_transform(df_three[' Label']).astype(np.int64)


In [ ]:
#编码方式已经改变
df_three[' Label'].dtypes

In [ ]:
df_three[' Label'].value_counts()


In [ ]:
#SMOTE解决类不平衡
pd.Series(df_three[' Label']).value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(n_jobs=-1,sampling_strategy={1:200,10:200,11:200,7:10})

In [ ]:
X_three = df_three.drop([' Label'],axis=1)
Y_three = df_three[' Label']
X_train_three, X_test_three, Y_train_three, Y_test_three = train_test_split(X_three,Y_three, train_size = 0.8, test_size = 0.2, random_state = 42) #shuffle=False

In [ ]:
#X_train_three, Y_train_three = smote.fit_resample(X_train_three, Y_train_three)

4.0

In [ ]:
# 热力分析（降维用的，具体不了解）
f,ax = plt.subplots(figsize=(24, 24))
corr = X.corr()
sns.heatmap(corr, annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.savefig('heatmap.png')

In [ ]:
import numpy as np

In [ ]:
# 降维
columns = np.full((corr.shape[0],), True, dtype=bool)
to_drop = []
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.75:
            if columns[j]:
                columns[j] = False
                to_drop.append((corr.columns)[j])
print('删除高度相关的特征：', to_drop)
X = X.drop(to_drop,axis = 1)
print(X.shape)

In [ ]:
# 降维
def RemoveMissing(train):
    train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
    #train_missing.head()
    train_missing = train_missing.index[train_missing > 0.75]
    all_missing = list(set(train_missing))
    train.drop(all_missing,axis = 1)
    print('There are %d columns with more than 75%% missing values' % len(all_missing))
    return train
X = RemoveMissing(X)

In [ ]:
# remove columns with only 1 unique value
# 如果某特征只有一个值，删除该特征
def remove_single_unique_values(dataframe):
    # 得到每一列唯一数值的数量
    cols_to_drop = dataframe.nunique()
    print(cols_to_drop)
    # 得到每一列唯一数值为1个的列标签
    cols_to_drop = cols_to_drop.loc[cols_to_drop.values==1].index
    print(cols_to_drop)
    print('There are %d columns with only 1 unique value' % len(cols_to_drop))
    # 删除唯一数值的特征
    dataframe = dataframe.drop(cols_to_drop,axis=1)
    return dataframe

X = remove_single_unique_values(X)

In [ ]:
print(X.shape)

In [ ]:
# 保存一下处理后的数据
X.to_csv('HandleData_X_processed.csv')

In [ ]:

filename = "HandleData_X_processed.csv"  # 文件名

X = pd.read_csv(filename)


In [ ]:
X = X.drop(['Unnamed: 0'],axis=1)

In [ ]:
print(X.info())
print(Y[" Label"].unique())

In [ ]:
print(Y[' Label'].unique())

In [ ]:
# 对Label进行编码
Y[' Label'].replace(to_replace=['BENIGN',
                'PortScan',
                'DDoS',
                'DoS slowloris',
                'DoS Slowhttptest',
                'DoS Hulk',
                'DoS GoldenEye',
                'Heartbleed',
                'FTP-Patator',
                'SSH-Patator',
                'Web Attack � Brute Force',
                'Web Attack � XSS',
                'Web Attack � Sql Injection',
                'Infiltration',
                'Bot'], 
            value=[ 0
                    ,1
                    ,2
                    ,3
                    ,4
                    ,5
                    ,6
                    ,7
                    ,8
                    ,9
                    ,10
                    ,11
                    ,12
                    ,13
                    ,14],
            inplace=True)

In [ ]:
print(Y[' Label'].unique())

In [ ]:
print(Y.info())

In [ ]:
# 无穷值或者空值处理
X.fillna(0, inplace=True)
train_X = np.isinf(X)
X[train_X]=0

In [ ]:
print(train_X.columns)

In [ ]:
print(X.info())

In [ ]:
X.columns

In [ ]:
# 调用模型进行训练，换其他模型改这里就行
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=12, min_samples_leaf=1, splitter="best")
trained_model = clf.fit(X,Y)
print("Score:", trained_model.score(X,Y))

In [ ]:
#调用SVM模型训练
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# 建立模型
model_svc = SVC(kernel='linear')
#训练模型
train_model_svc = model_svc.fit(X,Y)
# 计算准确率
print("Score:", train_model_svc.score(X,Y))

In [ ]:
#调用随机森林模型训练
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# 定义随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=123)
# 在训练集上拟合模型
train_model_rf = rf_model.fit(X, Y)
# 计算准确率
print("Score:", rf_model.score(X,Y))

In [ ]:
#调用KNN模型训练
from sklearn.neighbors import KNeighborsClassifier
# 创建一个 KNN 分类器对象
knn_model = KNeighborsClassifier(n_neighbors=3)
# 在训练集上拟合模型
train_model_knn = knn_model.fit(X, Y)
# 计算准确率
print("Score:", knn_model.score(X,Y))

In [ ]:
#神经网络
from sklearn.neural_network import MLPClassifier


In [ ]:
# 模型保存
import pickle
with open('clf.pickle','wb')as f: 
    pickle.dump(clf,f)
# 模型加载
with open('clf.pickle', 'rb') as f:
   clf2 = pickle.load(f)

In [ ]:
# 随机森林模型保存
import pickle
with open('rf.pickle','wb')as f: 
    pickle.dump(rf_model,f)
# 模型加载
with open('rf.pickle', 'rb') as f:
   rf_model2 = pickle.load(f)

In [ ]:
# knn模型保存
import pickle
with open('knn.pickle','wb')as f: 
    pickle.dump(knn_model,f)
# 模型加载
with open('knn.pickle', 'rb') as f:
   knn_model2 = pickle.load(f)

In [ ]:
# 读取待预测数据
mydata0 = pd.read_csv('Mydata/202211280951-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata2 = pd.read_csv('Mydata/20221128-10.19.2.20-0-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata3 = pd.read_csv('Mydata/20221128-10.19.2.20-1-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata4 = pd.read_csv('Mydata/20221128-10.19.2.20-2-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata5 = pd.read_csv('Mydata/20221128-10.19.2.20-3-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata6 = pd.read_csv('Mydata/20221128-10.19.2.20-4-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata7 = pd.read_csv('Mydata/20221128-10.19.3.44-0-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata8 = pd.read_csv('Mydata/20221128-10.19.3.44-1-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata9 = pd.read_csv('Mydata/20221128-10.19.3.44-2-ens224.pcap_Flow.csv', low_memory = False,encoding='gbk')
mydata10 = pd.read_csv('Mydata/202211280943.pcap_Flow.csv', low_memory = False,encoding='gbk')

mydata1= pd.concat([mydata0,mydata2,mydata3,mydata4,mydata5,mydata6,mydata7,mydata8,mydata9,mydata10])
mydata1.to_csv('Y_combine.csv')
#mydata1 = mydata1.rename(columns={' act data pkt fwd':'Fwd Act Data Pkts'})
print(mydata1.info())
X_test = mydata1.loc[:,mydata1.columns != " Label"]

In [ ]:
X_fs.columns

In [ ]:
# 训练时用到的特征值
imps = [' Average Packet Size', 
        ' Packet Length Mean', 
        ' Packet Length Std',
       ' Packet Length Variance', 
       ' Total Length of Bwd Packets',
       'Total Length of Fwd Packets', 
       ' Subflow Fwd Bytes',
       ' Subflow Bwd Bytes', 
       ' Bwd Packet Length Mean',
       ' Avg Bwd Segment Size',
        'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward',
        'Bwd Packet Length Max',
       ' Max Packet Length', 
       ' Fwd Packet Length Max',
        ' Fwd Header Length.1',
       ' Fwd Header Length', 
       ' Flow IAT Max', 
       ' Destination Port',
       ' Flow Duration',
        ' Bwd Header Length', 
        ' Fwd IAT Max',
         'Flow Bytes/s',
       'Fwd IAT Total', 
       ' Bwd Packets/s', 
       'Fwd Packets/s', 
       ' Flow Packets/s',
       ' Fwd IAT Mean', 
       ' Flow IAT Mean', 
       ' Fwd Packet Length Mean',
       ' Avg Fwd Segment Size', ' Subflow Bwd Packets', ' Flow IAT Std',
       ' Bwd Packet Length Std', ' Total Backward Packets', ' Fwd IAT Std',
       'Subflow Fwd Packets', ' Total Fwd Packets', 'Bwd IAT Total',
       ' Bwd IAT Max', ' Bwd Packet Length Min', ' Fwd Packet Length Std',
       ' Bwd IAT Mean', ' min_seg_size_forward', ' Bwd IAT Min',
       ' act_data_pkt_fwd', ' Fwd IAT Min', ' Min Packet Length', 'Idle Mean']

In [ ]:
# 取出对应特征值的数据并且进行处理
X_test = X_test.loc[:, X.columns]
test_X = np.isinf(X_test)
X_test[test_X]=0
X_test.fillna(0, inplace=True)

In [ ]:
print(X_test.info())

In [ ]:
X_test.columns

In [ ]:
# 预测
y_pred = knn_model.predict(X_test)

In [ ]:
print(y_pred)

In [ ]:
# 查看数据种类
ydata1_1 = pd.DataFrame(y_pred)
print(ydata1_1[0].unique())

In [ ]:
label_name = ['BENIGN',
                'PortScan',
                'DDoS',
                'DoS slowloris',
                'DoS Slowhttptest',
                'DoS Hulk',
                'DoS GoldenEye',
                'Heartbleed',
                'FTP-Patator',
                'SSH-Patator',
                'Web Attack � Brute Force',
                'Web Attack � XSS',
                'Web Attack � Sql Injection',
                'Infiltration',
                'Bot']

In [ ]:
# 对编码后的标签值进行还原
ydata1_0 = pd.DataFrame(ydata1_1)
mydata1['Label'] = ydata1_0
mydata1['Label'].replace(to_replace=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], value=label_name, inplace=True)
print(mydata1['Label'].unique())

In [ ]:
# 查看分布
def ClassDistribution(y):
    ax = sns.countplot(y[0],label="Count")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    print(y.groupby(0).size())

ClassDistribution(ydata1_1)